In [20]:
import numpy as np
import pandas as pd

import bqplot.pyplot as bplt
import ipywidgets as widgets

from collections import defaultdict
from matplotlib import pyplot as plt

import time

pd.set_option("display.max_columns", 50)

In [21]:
class TwoJarProblem:
    capacity1 = 0
    capacity2 = 0
    target = 0
    visitedDic = defaultdict(lambda: False)
    renderList = []
    waterLevel1 = []
    waterLevel2 = []

    def __init__(self, capacity1, capacity2, target):
        self.capacity1 = capacity1
        self.capacity2 = capacity2
        self.target = target
        self.solve(capacity1, capacity2)
        self.display()

    #         x,y
    # initially assume x = capacity1 and y = capacity2

    '''
        c1 = 3, c2 = 4 , t = 2
          initially x = c1, initially x2 = c2
          
          (3,4)  (3,0) (0,3) (3,3) (4,2)
          
          (0,4)
          (3,1)
          
          
    '''

    # def render_diplay(self,y1,y2):
    #     plt.bar([" jug1", " jug2"], [y1, y2])
    #     plt.show()

    def display(self):

        return self.renderList
        # print(self.renderList)
        # for i,j in self.renderList:
        #     self.render_diplay(i,j)

    def solve(self, x, y):
        if (x == self.target) or y == self.target:
            self.renderList.append((x, y))
            self.waterLevel1.append(x)
            self.waterLevel2.append(y)
            return True

        else:
            if self.visitedDic[(x, y)] == False:
                self.waterLevel1.append(x)
                self.waterLevel2.append(y)
                self.renderList.append((x, y))
                self.visitedDic[(x, y)] = True

                return (self.solve(0, y) or
                        self.solve(x, 0) or
                        self.solve(self.capacity1, y) or
                        self.solve(x, self.capacity2) or

                        self.solve(x + min(y, (self.capacity1 - x)),
                                   y - min(y, (self.capacity1 - x))) or
                        self.solve(x - min(x, (self.capacity2 - y)),
                                   y + min(x, (self.capacity2 - y))))
            else:
                return False

j1, j2, target = (3, 4, 2)
jar = TwoJarProblem(j1, j2, target)

In [22]:
def bfs(start, end, x_capacity, y_capacity):
	path = []
	front = []
	front.append(start)
	visited = []
	while(not (not front)):
		current = front.pop()
		x = current[0]
		y = current[1]
		path.append(current)
		if x == end or y == end:
			print("The solution is:")
			return path
			
		# rule 1
		if current[0] < x_capacity and ([x_capacity, current[1]] not in visited):
			front.append([x_capacity, current[1]])
			visited.append([x_capacity, current[1]])

		# rule 2
		if current[1] < y_capacity and ([current[0], y_capacity] not in visited):
			front.append([current[0], y_capacity])
			visited.append([current[0], y_capacity])

		# rule 3
		if current[0] > x_capacity and ([0, current[1]] not in visited):
			front.append([0, current[1]])
			visited.append([0, current[1]])

		# rule 4
		if current[1] > y_capacity and ([x_capacity, 0] not in visited):
			front.append([x_capacity, 0])
			visited.append([x_capacity, 0])

		# rule 5
		#(x, y) -> (min(x + y, x_capacity), max(0, x + y - x_capacity)) if y > 0
		if current[1] > 0 and ([min(x + y, x_capacity), max(0, x + y - x_capacity)] not in visited):
			front.append([min(x + y, x_capacity), max(0, x + y - x_capacity)])
			visited.append([min(x + y, x_capacity), max(0, x + y - x_capacity)])

		# rule 6
		# (x, y) -> (max(0, x + y - y_capacity), min(x + y, y_capacity)) if x > 0
		if current[0] > 0  and ([max(0, x + y - y_capacity), min(x + y, y_capacity)] not in visited):
			front.append([max(0, x + y - y_capacity), min(x + y, y_capacity)])
			visited.append([max(0, x + y - y_capacity), min(x + y, y_capacity)])

	return "Not found"

def gcd(a, b):
	if a == 0:
		return b
	return gcd(int(b%a), int(a))


In [23]:
end = 5
x_capacity = 7
y_capacity = 13
final_path = bfs([0,0], end, x_capacity, y_capacity)
if end % gcd(x_capacity,y_capacity) == 0:
	for i in range(0,len(final_path)):
		print(final_path[i])
else:
	print("No solution possible for this combination.")

The solution is:
[0, 0]
[0, 4]
[3, 1]
[3, 4]
[3, 0]
[0, 3]
[3, 3]
[2, 4]


In [25]:
# Chart Creation Logic
# heights = [10, 20, 30, 40, 50, 60, 70]
heights = [(x * 10, y * 10) for (x, y) in final_path]

fig = bplt.figure(animation_duration=1000,
                  title="Water Jug Problem",
                  fig_margin={'top':50, 'bottom':50, 'left':150, 'right':150},
                  background_style={'fill':"lightgray"})

bar = bplt.bar(x=[1,2],y=heights)
bar.y = (0, 0)

bplt.xlabel("X")
bplt.ylabel("Y")

bplt.ylim(0,75);

# Callback to Update Chart
def update_bar_chart(btn):
    for i in heights:
        # idx = np.random.choice(np.arange(len(colors))) # Choose random color
        # np.random.shuffle(heights) # Shuffle heights
        with bar.hold_sync(): # updating scatter data
            bar.y = i
            # bar.colors = [colors[idx]]
        time.sleep(3)

btn = widgets.Button(description="Start Anime", icon="play")

btn.on_click(update_bar_chart)

widgets.VBox([fig, btn])